In [1]:
#NOTE: use paimg1 env, the retccl one has package issue with torchvision
import sys
import os
import numpy as np
import openslide
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')
import pandas as pd
import warnings
import torch
import torch.nn as nn

from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader
import torch.optim as optim
from pathlib import Path
from scipy.spatial.distance import cdist


sys.path.insert(0, '../Utils/')
from Utils import create_dir_if_not_exists
from Utils import generate_deepzoom_tiles, extract_tile_start_end_coords, get_map_startend
from Utils import get_downsample_factor
from Utils import minmax_normalize, count_label
from Utils import log_message, set_seed
from Eval import compute_performance, plot_LOSS, compute_performance_each_label, get_attention_and_tileinfo
from train_utils import pull_tiles, get_feature_label_array_dynamic
from train_utils import ModelReadyData_diffdim, convert_to_dict, prediction
from Model import Mutation_MIL_MT
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
####################################
######      USERINPUT       ########
####################################
TRAIN_SAMPLE_SIZE = "ALLTUMORTILES"
TRAIN_OVERLAP = 100
TEST_OVERLAP = 0
SELECTED_FOLD = 0
CLUSTER_ALG = 'KMEAN'
N_CLUSTERS = 4
CLUSTER_DIST = 'L2'
TUMOR_FRAC_THRES = 0
INCLUDE_TF = True
INCLUDE_CLUSTER = True

##################
###### DIR  ######
##################
proj_dir = '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/'
feature_extraction_method = 'retccl'

indata_path = proj_dir + 'intermediate_data/model_ready_data/feature_' + feature_extraction_method + '/MAXSS'+ str(TRAIN_SAMPLE_SIZE)  + '_TrainOL' + str(TRAIN_OVERLAP) +  '_TestOL' + str(TEST_OVERLAP) + '_TFT' + str(TUMOR_FRAC_THRES) + "/split_fold" + str(SELECTED_FOLD) + "/"
data_info_path = indata_path + "init_data_info/"
updated_feature_path = indata_path + "clusters/" + CLUSTER_ALG + "/"
updated_feature_folder_name = "_NCLUSTER_" + str(N_CLUSTERS) +  "_DISTMETRIC_" + CLUSTER_DIST 

################################################
#Create output dir
################################################
outdir0 =  indata_path + "model_input/"
create_dir_if_not_exists(outdir0)

if INCLUDE_TF == False and INCLUDE_CLUSTER == False:
    outdir =  outdir0 + "emb_only/"
elif INCLUDE_TF == True and INCLUDE_CLUSTER == False:
    outdir =  outdir0 + "emb_and_tf/"
elif INCLUDE_TF == False and INCLUDE_CLUSTER == True:
    outdir =  outdir0 + "emb_and_cluster/"
elif INCLUDE_TF == True and INCLUDE_CLUSTER == True:
    outdir =  outdir0 + "emb_and_tf_and_cluster/"

create_dir_if_not_exists(outdir)
##################
#Select GPU
##################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
set_seed(0)

Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/model_ready_data/feature_retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0/split_fold0/model_input/' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/model_ready_data/feature_retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0/split_fold0/model_input/emb_and_tf_and_cluster/' created.
cuda:0


In [ ]:
############################################################################################################
#Get features and labels
############################################################################################################
train_feature = torch.load(updated_feature_path + 'updated_train_feature' + updated_feature_folder_name + '.pth')
test_feature = torch.load(updated_feature_path + 'updated_test_feature' + updated_feature_folder_name + '.pth')
val_feature = torch.load(updated_feature_path + 'updated_val_feature' + updated_feature_folder_name+ '.pth')

train_label = torch.load(data_info_path + 'train_label.pth')
test_label = torch.load(data_info_path + 'test_label.pth')
val_label = torch.load(data_info_path + 'val_label.pth')

train_info = torch.load(data_info_path + 'train_info.pth')
test_info = torch.load(data_info_path + 'test_info.pth')
val_info = torch.load(data_info_path + 'val_info.pth')

train_tf_info = torch.load(data_info_path + 'train_tf_info.pth')
test_tf_info = torch.load(data_info_path + 'test_tf_info.pth')
val_tf_info = torch.load(data_info_path + 'val_tf_info.pth')


train_ids = torch.load(data_info_path + 'train_ids.pth')
test_ids = torch.load(data_info_path + 'test_ids.pth')
val_ids = torch.load(data_info_path + 'val_ids.pth')

In [ ]:
################################################
#     Model ready data 
################################################
train_data = ModelReadyData_diffdim(train_feature,train_label,train_tf_info,include_tumor_fraction = INCLUDE_TF, include_cluster = INCLUDE_CLUSTER)
test_data = ModelReadyData_diffdim(test_feature,test_label,test_tf_info,include_tumor_fraction = INCLUDE_TF, include_cluster = INCLUDE_CLUSTER)
val_data = ModelReadyData_diffdim(val_feature,val_label,val_tf_info,include_tumor_fraction = INCLUDE_TF, include_cluster = INCLUDE_CLUSTER)

In [ ]:
#Output
torch.save(train_data, outdir + 'train_data.pth')
torch.save(test_data,  outdir + 'test_data.pth')
torch.save(val_data,   outdir + 'val_data.pth')

torch.save(train_info,   outdir + 'train_info.pth')
torch.save(test_info,   outdir + 'test_info.pth')
torch.save(val_info,   outdir + 'val_info.pth')

torch.save(train_tf_info,   outdir + 'train_tf_info.pth')
torch.save(test_tf_info,   outdir + 'test_tf_info.pth')
torch.save(val_tf_info,   outdir + 'val_tf_info.pth')


torch.save(train_ids,   outdir + 'train_ids.pth')
torch.save(test_ids,   outdir + 'test_ids.pth')
torch.save(val_ids,   outdir + 'val_ids.pth')